In [1]:
PROJECT_ID = "lramsey-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
import tempfile
import os
from typing import List, Optional, Tuple, Dict
from common.utils.logging_handler import Logger
from common.models import (UserQuery, QueryResult, QueryEngine, QueryDocument,
                           QueryReference, QueryDocumentChunk, BatchJobModel)
from common.utils.errors import (ResourceNotFoundException,
                                 ValidationError)
from common.utils.http_exceptions import InternalServerError
from utils.errors import NoDocumentsIndexedException
from google.cloud import storage
from services.query import query_prompts
from services.query.vector_store import VectorStore
from services.query.data_source import DataSource
    
from config import (PROJECT_ID, DEFAULT_QUERY_CHAT_MODEL,
                        DEFAULT_QUERY_EMBEDDING_MODEL)
from services.query.vertex_search import build_vertex_search

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  database "pgvector" does not exist

(Background on this error at: https://sqlalche.me/e/14/e3q8)
INFO: [utils/

In [4]:
storage_client = storage.Client(project=PROJECT_ID)

In [5]:
query_engine = "Test Vertex Search 3"
doc_url = "https://dmv.nv.gov/"
user_id = "epKVwPZeIBz1rrqcDm1B"
is_public=True
query_description="test vertex search"

In [6]:
from common.models.llm_query import QE_TYPE_VERTEX_SEARCH, QE_TYPE_LLM_SERVICE

# create q_engine
params = {"depth_limit": 1}
query_engine_type = QE_TYPE_VERTEX_SEARCH
embedding_type = DEFAULT_QUERY_EMBEDDING_MODEL
llm_type = DEFAULT_QUERY_EMBEDDING_MODEL
vector_store_type = None
q_engine = QueryEngine(name=query_engine,
                     created_by=user_id,
                     query_engine_type=QE_TYPE_VERTEX_SEARCH,
                     llm_type=llm_type,
                     description=query_description,
                     embedding_type=embedding_type,
                     vector_store=vector_store_type,
                     params=params,
                     doc_url=doc_url)
q_engine.save()

In [9]:
import re

In [11]:
from services.query.data_source import DataSource
bucket_name = DataSource.downloads_bucket_name(q_engine)

In [18]:
bucket_name = 'lramsey-dev-downloads-test-vertex-search-3'

In [19]:
re.fullmatch("^[a-z0-9][a-z0-9._-]{1,61}[a-z0-9]$", bucket_name)

<re.Match object; span=(0, 42), match='lramsey-dev-downloads-test-vertex-search-3'>

In [7]:
build_vertex_search(q_engine)

INFO: [query/vertex_search.py:174 - build_vertex_search()] Building vertex search engine [Test Vertex Search 3] [https://dmv.nv.gov/]
INFO: [query/vertex_search.py:445 - download_web_docs()] downloading web docs to bucket [lramsey-dev-downloads-Test-Vertex-Search-3]
ERROR: [query/vertex_search.py:221 - build_vertex_search()] Error building vertex search query engine [400 GET https://storage.googleapis.com/storage/v1/b/lramsey-dev-downloads-Test-Vertex-Search-3?fields=name&prettyPrint=false: Invalid bucket name: 'lramsey-dev-downloads-Test-Vertex-Search-3']
ERROR: [query/vertex_search.py:222 - build_vertex_search()] None
INFO: [query/vertex_search.py:410 - delete_vertex_search()] deleting vertex search query engine Test Vertex Search 3


Traceback (most recent call last):
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/services/query/vertex_search.py", line 190, in build_vertex_search
    gcs_url = download_web_docs(q_engine, data_url)
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/services/query/vertex_search.py", line 447, in download_web_docs
    web_data_source.download_documents(data_url, temp_dir)
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/services/query/web_datasource.py", line 238, in download_documents
    create_bucket(self.storage_client, self.bucket_name)
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/../src/utils/gcs_helper.py", line 42, in create_bucket
    if not bucket.exists():
  File "/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service

([], [])

In [8]:
from google.cloud import discoveryengine_v1alpha as discoveryengine
location = "global"
project_id = PROJECT_ID
parent = "projects/lramsey-dev/locations/global/collections/default_collection"

In [ ]:
from services.query.vertex_search import (create_data_store, 
                                          create_search_engine, 
                                          import_documents_to_datastore, 
                                          import_documents_gcs, wait_for_operation, datastore_id_from_name, inventory_gcs_files,
                                          wait_for_operation)

In [ ]:
data_store_id = datastore_id_from_name(q_engine.name)
data_store_id

In [ ]:
data_url = q_engine.doc_url
project_id = PROJECT_ID
location = "global"

docs_to_be_processed = []
docs_processed = []
docs_not_processed = []

In [ ]:
docs_to_be_processed = inventory_gcs_files(data_url)

In [ ]:
docs_to_be_processed

In [ ]:
operation = create_data_store(q_engine, project_id, data_store_id)
wait_for_operation(operation)

In [ ]:
client = discoveryengine.DocumentServiceClient()
parent = client.branch_path(
    project=project_id,
    location=location,
    data_store=data_store_id,
    branch="default_branch",
)

In [ ]:
data_url

In [ ]:
operation = import_documents_gcs(data_url,
                                 docs_to_be_processed,
                                 client,
                                 parent)

In [ ]:
print(f"Waiting for import operation to complete: {operation.operation.name}")

In [ ]:
wait_for_operation(operation)

In [ ]:
metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

In [ ]:
metadata

In [ ]:
operation.__dict__

In [ ]:
operation.result()

In [ ]:
!gsutil ls -R gs://944045413892_us_import_content/errors16429642499957673123

In [ ]:
if metadata.success_count == len(docs_to_be_processed):
  docs_processed = docs_to_be_processed
else:
  # TODO: build list of documents processed/not processed from results
  pass

In [ ]:
docs_processed

In [ ]:
operation = create_search_engine(q_engine, project_id, data_store_id)

In [ ]:
q_engine.index_id = data_store_id
q_engine.update()

In [ ]:
from services.query.vertex_search import query_vertex_search, perform_vertex_search

In [ ]:
search_query = "what qualifies as an encounter in medicaid billing?"

In [ ]:
client = discoveryengine.SearchServiceClient()

In [ ]:
serving_config = client.serving_config_path(
    project=project_id,
    location=location,
    data_store=data_store_id,
    serving_config="default_config",
)

In [ ]:
content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
    snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
        return_snippet=True
    ),
    summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
        summary_result_count=5,
        include_citations=True,
        ignore_adversarial_query=True,
        ignore_non_summary_seeking_query=True,
    ),
)

In [ ]:
request = discoveryengine.SearchRequest(
    serving_config=serving_config,
    query=search_query,
    page_size=10,
    content_search_spec=content_search_spec,
    query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
    ),
    spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
        mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
    ),
)

In [ ]:
response = client.search(request)

In [ ]:
len(response.results)

In [ ]:
results = response.results
type(results[0])

In [ ]:
type(response)

In [ ]:
document = results[0].document

In [ ]:
type(document)

In [ ]:
from google.protobuf.json_format import MessageToDict

In [ ]:
import proto

In [ ]:
document_dict = proto.Message.to_dict(document)

In [ ]:
document_dict.keys()

In [ ]:
document_dict["derived_struct_data"]["snippets"][0]["snippet"]

In [ ]:
document_dict["derived_struct_data"]

In [ ]:
document_data = proto.Message.to_dict(document)["derived_struct_data"]

In [ ]:
document_data["link"]

In [ ]:
document_data["snippets"][0]["snippet"]